# Clean and format Ademe data

Imports

In [ ]:
import pandas as pd

### Load and concat data

In [ ]:
existant = pd.read_csv('data/existant74.csv')
neuf = pd.read_csv('data/neuf74.csv')

Add `data_source` column to differenciate existings from new

In [ ]:
existant['data_source'] = 'existant'
neuf['data_source'] = 'existant'

Concat

In [ ]:
df = pd.concat([existant, neuf], ignore_index=True)

In [ ]:
print('existant', len(existant.columns))
print('neuf', len(neuf.columns))
print('df:', len(df.columns))

### Check NAs

In [ ]:
df.isna().sum()

### Add data from `communes-france-2025.csv`

Load communes and keep only interesting columns

In [ ]:
communes = pd.read_csv('../web/application/datasets/communes-france-2025.csv')
communes = communes[['code_insee', 'population', 'superficie_km2', 'densite', 'altitude_moyenne', 'grille_densite_texte']]

Concat data and check for non match

In [ ]:
merged_df = pd.merge(df, communes, left_on='code_insee_ban', right_on='code_insee', how='left', indicator=True)
unmatched_rows = merged_df[merged_df['_merge'] == 'left_only']

### Fix non matched

Annecy makes a mess, little cities got grouped with Annecy, being now called CA Grand Annecy

In [ ]:
old_annecy_insees = unmatched_rows['code_insee_ban'].unique()
old_annecy_insees

Drop Montmin 74187 (318 peoples). , It's a new city created after the fusion. Replace `old_annecy_insees` with Annecy insee 74010

In [ ]:
df = df[df['code_insee_ban'] != 74187] # Drop Montmin
df['code_insee_ban'] = df['code_insee_ban'].replace(old_annecy_insees, 74010) # replace old insee to new insee

Remerge 

In [ ]:
merged_df = pd.merge(df, communes, left_on='code_insee_ban', right_on='code_insee', how='left')
merged_df = merged_df.drop(columns=['code_insee'])

### Fix `classe_altitude`

In [ ]:
def categoriser_altitude(altitude):
    """
    Affecte chaque logement à une tranche d'altitude
    
    Cette fonction permet de regrouper les logements selon leur altitude
    pour faciliter les analyses comparatives entre vallées et montagnes.
    
    Args:
        altitude (float): Altitude du logement en mètres
        
    Returns:
        str: Label de la tranche d'altitude correspondante
    """
    # Gestion des valeurs manquantes
    if pd.isna(altitude):
        return pd.NA
    
    # Attribution de la tranche selon les seuils définis
    if altitude < 600:
        return "0-600m (Vallée)"
    elif altitude < 1200:
        return "600-1200m (Colline)"
    elif altitude < 1800:
        return "1200-1800m (Montagne)"
    elif altitude < 2500:
        return "1800-2500m (Haute montagne)"
    else:
        return ">2500m (Très haute montagne)"

In [ ]:
merged_df['classe_altitude'] = merged_df['altitude_moyenne'].apply(categoriser_altitude)

### Add `categorie_dpe`

In [ ]:
def categoriser_dpe(etiquette):
    if etiquette in ['A', 'B']:
        return 'Bons (A-B)'
    elif etiquette in ['C', 'D']:
        return 'Moyens (C-D)'
    elif etiquette == 'E':
        return 'Médiocres (E)'
    else:  # F ou G
        return 'Passoires (F-G)'

In [ ]:
merged_df['categorie_dpe'] = merged_df['etiquette_dpe'].apply(categoriser_dpe)

### Add `passoire`

In [ ]:
merged_df['passoire'] = merged_df['etiquette_dpe'].isin(['F', 'G'])

### Add `periode_categorie`

In [ ]:
def categoriser_periode(periode):
    """Regroupe les périodes de construction en grandes époques"""
    if pd.isna(periode):
        return pd.NA
    
    periode_str = str(periode).lower()
    
    # Strings
    if any(x in periode_str for x in ['récent']):
        return 'Après 2012'
    elif any(x in periode_str for x in ['avant', 'ancien']):
        return 'Avant 1975'
    
    periode_int = int(periode_str.split('-')[0].split(' ')[-1])

    # Avant 1975 (première réglementation thermique)
    if periode_int < 1975:
        return 'Avant 1975' 
    # 1975-2000 (RT 1974, 1988, 2000)
    elif periode_int <= 2000:
        return '1975-2000'
    # 2001-2012 (RT 2005, 2012)
    elif periode_int <= 2012:
        return '2001-2012'
    # Après 2012 (RT 2012, RE 2020)
    else:
        return 'Après 2012'

In [ ]:
merged_df['periode_categorie'] = merged_df['periode_construction'].apply(categoriser_periode)

### Save CSV

In [ ]:
merged_df.to_csv('data/logements_74.csv', index=False)
merged_df.shape